In [123]:
pip install category_encoders

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [124]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer 

from category_encoders import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

## 데이터 불러오기

In [125]:
df = pd.read_csv('/Users/yeseulseo/Section3/project/csv/서울시 부동산 실거래가 정보.csv', encoding='cp949')
df

/var/folders/yp/r9q2z09j2rj4vfhr9q8r9vl40000gn/T/ipykernel_1142/1303832897.py:1: DtypeWarning: Columns (6,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/yeseulseo/Section3/project/csv/서울시 부동산 실거래가 정보.csv', encoding='cp949')


,접수연도,자치구코드,자치구명,법정동명,법정동코드,지번구분명,본번,부번,건물명,계약일,물건금액(만원),건물면적(㎡),토지면적(㎡),층,권리구분,취소일,건축년도,건물용도,신고구분,신고한 개업공인중개사 시군구명
0,2022,11530,구로구,구로동,10200,대지,106.0,2.0,대림역포스큐,20220623,11200,19.02,28.60,5.0,NaN,NaN,2015.0,오피스텔,중개거래,서울 구로구
1,2022,11740,강동구,천호동,10900,대지,425.0,3.0,천호역 한강 푸르지오시티,20220623,18000,25.24,35.91,10.0,NaN,NaN,2015.0,오피스텔,중개거래,경기 하남시
2,2022,11470,양천구,신월동,10300,대지,143.0,10.0,라온마브빌,20220623,23500,29.11,19.90,4.0,NaN,NaN,2019.0,연립다세대,중개거래,서울 강서구
3,2022,11500,강서구,화곡동,10300,대지,346.0,74.0,리빙하우스,20220623,22000,27.75,18.00,4.0,NaN,NaN,2016.0,연립다세대,중개거래,서울 강서구
4,2022,11500,강서구,화곡동,10300,대지,424.0,29.0,에스엠파크빌,20220623,29800,36.24,22.17,3.0,NaN,NaN,2015.0,연립다세대,중개거래,서울 강서구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639995,2018,11710,송파구,잠실동,10100,일반,35.0,0.0,트리지움,20180103,142500,84.83,0.00,23.0,NaN,NaN,2007.0,아파트,NaN,NaN
639996,2018,11410,서대문구,북아현동,11000,일반,1.0,954.0,힐스테이트 신촌,20180103,67775,59.70,0.00,16.0,입주권,NaN,0.0,아파트,NaN,NaN
639997,2018,11230,동대문구,전농동,10400,NaN,NaN,NaN,NaN,20180103,56000,177.44,96.00,NaN,NaN,NaN,1996.0,단독다가구,NaN,NaN
639998,2018,11320,도봉구,창동,10700,일반,623.0,89.0,이화빌라B,20180103,12500,33.12,26.49,3.0,NaN,NaN,1991.0,연립다세대,NaN,NaN


In [126]:
df.columns



Index(['접수연도', '자치구코드', '자치구명', '법정동명', '법정동코드', '지번구분명', '본번', '부번', '건물명',
       '계약일', '물건금액(만원)', '건물면적(㎡)', '토지면적(㎡)', '층', '권리구분', '취소일', '건축년도',
       '건물용도', '신고구분', '신고한 개업공인중개사 시군구명'],
      dtype='object')

## EDA

In [127]:
df.isna().sum()

접수연도                     0
자치구코드                    0
자치구명                     0
법정동명                     0
법정동코드                    0
지번구분명                48627
본번                   48616
부번                   48616
건물명                  48628
계약일                      0
물건금액(만원)                 0
건물면적(㎡)                  0
토지면적(㎡)             159177
층                    48585
권리구분                634040
취소일                 624526
건축년도                  2524
건물용도                     0
신고구분                592855
신고한 개업공인중개사 시군구명    605170
dtype: int64

In [128]:
df.nunique()

접수연도                    6
자치구코드                  25
자치구명                   25
법정동명                  420
법정동코드                  86
지번구분명                   6
본번                   3515
부번                   1292
건물명                 57971
계약일                  1633
물건금액(만원)            14068
건물면적(㎡)             28353
토지면적(㎡)             12550
층                      76
권리구분                    2
취소일                   783
건축년도                  106
건물용도                    4
신고구분                    2
신고한 개업공인중개사 시군구명      620
dtype: int64

### 사용할 컬럼만 선택

In [129]:
df1=df[['자치구명', '법정동명','물건금액(만원)','층','건물용도']]
df1.head()

,자치구명,법정동명,물건금액(만원),층,건물용도
0,구로구,구로동,11200,5.0,오피스텔
1,강동구,천호동,18000,10.0,오피스텔
2,양천구,신월동,23500,4.0,연립다세대
3,강서구,화곡동,22000,4.0,연립다세대
4,강서구,화곡동,29800,3.0,연립다세대


In [130]:
df1.isna().sum()

자치구명            0
법정동명            0
물건금액(만원)        0
층           48585
건물용도            0
dtype: int64

In [131]:
#지하는 제외
df1=df1.query("층 >=0 ")
df1.head()

,자치구명,법정동명,물건금액(만원),층,건물용도
0,구로구,구로동,11200,5.0,오피스텔
1,강동구,천호동,18000,10.0,오피스텔
2,양천구,신월동,23500,4.0,연립다세대
3,강서구,화곡동,22000,4.0,연립다세대
4,강서구,화곡동,29800,3.0,연립다세대


In [132]:
df1.isna().sum()


자치구명        0
법정동명        0
물건금액(만원)    0
층           0
건물용도        0
dtype: int64

In [133]:
df2=df1.sample(frac=0.04)
df2.shape

(22976, 5)

#### train,test 나누기

In [134]:
target='물건금액(만원)'
train, test = train_test_split(df2, test_size=0.2, random_state=42)

print(train.shape,  test.shape)

(18380, 5) (4596, 5)


In [135]:
features = train.columns.drop([target])
X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]
X_train.head()

,자치구명,법정동명,층,건물용도
317845,송파구,문정동,15.0,오피스텔
479863,송파구,오금동,23.0,아파트
168885,강서구,화곡동,16.0,아파트
170149,송파구,거여동,8.0,아파트
151366,서초구,반포동,29.0,아파트


In [136]:
X_train['층']

317845    15.0
479863    23.0
168885    16.0
170149     8.0
151366    29.0
          ... 
401087     8.0
168227     4.0
204819     4.0
224980     6.0
345365     3.0
Name: 층, Length: 18380, dtype: float64

## RandomForest를 사용한 모델링
### 자치구명, 법정동명, 건물용도 컬럼들은 OrdinalEncoder 사용

In [138]:
cat_features = ['자치구명', '법정동명', '건물용도']
model = Pipeline([
    ('ordinal', OrdinalEncoder(cols=cat_features)),
    ('rf', RandomForestClassifier(n_estimators=200, max_depth=10,random_state=42))
])

model.fit(X_train, y_train)

Pipeline(steps=[('ordinal',
                 OrdinalEncoder(cols=['자치구명', '법정동명', '건물용도'],
                                mapping=[{'col': '자치구명',
                                          'data_type': dtype('O'),
                                          'mapping': 송파구      1
강서구      2
서초구      3
동작구      4
영등포구     5
노원구      6
양천구      7
강남구      8
강동구      9
광진구     10
성동구     11
은평구     12
종로구     13
중랑구     14
마포구     15
성북구     16
도봉구     17
강북구     18
동대문구    19
용산구     20
금천구     21
중구      22
관악구     23
구로구     24
서대문구    25
NaN     -2
dtype: int64},
                                         {'col': '법정동명',
                                          'data_type': dtype('O'),
                                          'mapping': 문정동        1
오금동        2
화곡동        3
거여동        4
반포동        5
        ... 
익선동      337
장충동1가    338
가회동      339
인현동2가    340
NaN       -2
Length: 341, dtype: int64},
                                         {'col': '건물용도',
                                          'data_type': dtype('O'),
                                          'mapping': 오피스텔     1
아파트      2
연립다세대    3
NaN     -2
dtype: int64}])),
                ('rf',
                 RandomForestClassifier(max_depth=10, n_estimators=200,
                                        random_state=42))])

## 모델을 pkl사용하여 저장 

In [142]:
import joblib
joblib.dump(model, 'model.pkl')

['model.pkl']

In [141]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22976 entries, 247900 to 242050
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   자치구명      22976 non-null  object 
 1   법정동명      22976 non-null  object 
 2   물건금액(만원)  22976 non-null  int64  
 3   층         22976 non-null  float64
 4   건물용도      22976 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 1.1+ MB
